<a href="https://colab.research.google.com/github/Isaquintero2004/Anal-tica-de-Negocios-2/blob/main/RETO_N%C2%B02_ISABELA_QUINTERO_FERN%C3%81NDEZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RETO N°2: ÁRBOLES DE DECISIÓN**
De acuerdo con la Base de Datos Tech Sales Report, implementar un modelo basado en los principios de los Árboles de Decisión para determinar si un empleado posee título universitario o no (College: Yes – 1, No - 0), tomando como referencia un total 8 variables socioeconómicas sin incluir la variable personalidad. Para la variable de "Business" tomar como Hardware:0 y para Software: 1.

De acuerdo con lo anterior, llevar a cabo la caracterización de la base de datos, total de personas, número de personas inscritas y número de personas no escritas.

0. Se procede con la carga de librerias de trabajo

In [ ]:
import numpy as np #para datos númericos, raíces y demás
import pandas as pd #comunicación con el excel

#Para los árboles de decisión: implementación y evaluación del modelo
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Para la carga automática de los archivos desde el Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se procede con la carga de la base de datos

In [ ]:
#Cargamos la base de datos
nxl="/content/drive/MyDrive/ANALÍTICA DE NEGOCIOS/2. TechSalesReport_Int_L.xlsx"
XDB=pd.read_excel(nxl, sheet_name=0) #usamos la hoja donde está toda la información
XDB.head(100)  #solo muestra 100 datos

#Convertimos la variable "Business" a valores numéricos
XDB["Business"] = XDB["Business"].map({"Hardware": 0, "Software": 1})

#Convertimos la variable "College" a valor numérico
XDB["College"] = XDB["College"].map({"No": 0, "Yes": 1})

#Se cargan las variables de interés
XDB=XDB[["Business","Age","Female","Years", "College", "Certificates", "Feedback", "Salary", "NPS"]]

#Seleccionamos las variables
XD=XDB[["Business","Age","Female","Years", "Certificates", "Feedback", "Salary", "NPS"]] #Variables de entrada
yd=XDB[["College"]] #Variable de salida o referencia
XDB

#Caracterización de la base de datos
#Total de personas
total_personas = len(XDB)
print("Total de personas:", total_personas)

#Número de personas inscritas (College = 1)
inscritos = XDB['College'].sum()
print("Número de personas inscritas:", inscritos)

#Número de personas no inscritas (College = 0)
no_inscritos = total_personas - inscritos
print("Número de personas no inscritas:", no_inscritos)


Total de personas: 21990
Número de personas inscritas: 17520
Número de personas no inscritas: 4470


2. Se procede a la implementación del modelo

*   max_depth = Número de variables disponibles
*   gini = criterio que indica el grado de error del modelo en la clasificación de solicitantes

In [ ]:
#Se llama al árbol
mar=DecisionTreeClassifier(criterion="gini", max_depth=4)
mar.fit(XD, yd)

#Se procede con el despliegue del árbol
from sklearn.tree import export_graphviz #exportar archivo
from pydotplus import graph_from_dot_data #Crear el gráfico

#Nombramos las variables
vs=["Business","Age","Female","Years", "Certificates", "Feedback", "Salary", "NPS"] #Etiquetar el árbol (nombrarlo)

#Como quiero que me exporte el archivo
dot_data=export_graphviz(mar, feature_names=vs) #Exportarlo con los nombres
graph=graph_from_dot_data(dot_data) #Crear el gráfico
graph.write_png("ArbolCollege.png")

True

3. Desempeño del modelo

In [ ]:
#Matriz de confusión
ydp=mar.predict(XD)
cm=confusion_matrix(yd, ydp)
print(cm)

VN=cm[0,0]; FP=cm[0,1]; FN=cm[1,0]; VP=cm[1,1]

#Tasa de exactitud
Ex=(VN+VP)/(VP+VN+FP+FN) #En cuanto acierta
print("La exactitud es:", Ex)

#Tasa sensibilidad
Sen=VP/(VP+FN) #Como se comportó el modelo con los positivos (con título universitario)
print("La sensibilidad es:", Sen)

#Tasa especificidad
Esp=VN/(VN+FP) #Como se comportó el modelo con los negativos (sin título universitario)
print("La especificidad es:", Esp)

#Tasa de precisión
Pre=VP/(VP+FP) #Cual es el acierto clasificando los positivos
print("La precisión es:", Pre)

#Tasa de error
TE=(FP+FN)/(VP+VN+FP+FN) #En cuanto se equivoca
print("La tasa de error es:", TE)

#Tasa de precisión negativa
PredNeg=VN/(VN+FN) #Como funciona obteniendo los negativos
print("La precisión negativa es:", PredNeg)

[[  114  4356]
 [   69 17451]]
La exactitud es: 0.7987721691678036
La sensibilidad es: 0.9960616438356165
La especificidad es: 0.025503355704697986
La precisión es: 0.8002476269087908
La tasa de error es: 0.20122783083219645
La precisión negativa es: 0.6229508196721312


4. Realizamos el pronóstico

In [ ]:
#PRONOSTICO
#Tomamos la segunda hoja de pronóstico
XDB2=pd.read_excel(nxl, sheet_name=1) #usamos la hoja pronóstico
XDB2.head(100)  #solo muestra 100 datos

#Eliminamos columna Personality
XDB2=XDB2.drop(["Personality"], axis=1)

#Convertimos la variable "Business" a valores numéricos
XDB2["Business"] = XDB2["Business"].map({"Hardware": 0, "Software": 1})

#Crear la columna "College" con valores NaN
XDB2["College"] = np.nan

#Se hace el ponóstico de College para estas personas
XD2=XDB2[["Business","Age","Female","Years", "Certificates", "Feedback", "Salary", "NPS"]]

#Se hace el pronóstico para estas personas como tal:
ydp2=mar.predict(XD2)
print("Las predicciones son: \n", ydp2)

#Asignar las predicciones a la columna "College"
XDB2["College"] = ydp2

#Crear el informe con College
dfp = pd.DataFrame(XDB2)

#Colocar nombre a columnas
dfp.columns=[["Sales_Rep", "Business","Age","Female","Years", "Certificates", "Feedback", "Salary", "NPS", "College"]]
dfp.head(10)

#Crear un excel descargable
dfp.to_excel("InformeCollege.xlsx") #Aparece en la carpeta content
dfp.head(10)

#Contar cuantos son negativos y cuanto son positivos
print("Los que no tienen título universitario pronosticado son:", len(np.where(ydp2==0)[0]))
print("Los que tienen título universitario pronosticado son:", len(np.where(ydp2==1)[0]))


Las predicciones son: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Los que no tienen título universitario pronosticado son: 0
Los que tienen título universitario pronosticado son: 16


De acuerdo con los resultados arrojados por el modelo, se pudo corrobar en la plataforma colab el comportamiento del modelo de árboles de decisión de acuerdo con el archivo de excel de referencia acerca del reporte de ventas realizados por una empresa de alta tecnología.

# **Descripción de las variables**
Para comprender el proceso realizado, primero es importante tener una idea sobre la base de datos con la que se trabajó y sus variables:

**"Business"** hace referencia al tipo de producto que se vendió por cada empleado, es decir, los productos ofrecidos por la empresa de tecnología, para este caso se contaban con dos opciones, la venta de hardware que fue reemplazado por el número 0 y software que fue reemplazado con el número 1, estos cambios con la finalidad de llevar a cabo de manera exitosa el pronóstico, **"Age"** hace referencia a la edad de los empleados, contiene una variedad de números que indican la edad de cada uno que vienen desde el empleado menor con 21 años hasta el mayor con 65 años, en promedio las edades rondan entre los 42 años, **"Female"** es una variable que indica el género del empleado, dividido en dos opciones, aquellos con el número 1 son mujeres y aquellos con el número 0 son el caso contrario, es decir, hombres, **"Years"** indica el número de años que el empleado ha estado en la empresa, hay empleados que apenas están en su primer año con la empresa, mientras que hay otros que llevan más de 12 años en la misma, **"Certificates"** hace referencia al número de certiﬁcaciones profesionales relevantes que ha obtenido cada empleado, encontrándose valores desde 0 (ninguna certificación) hasta 6 certificaciones dentro de la base de datos, **"Feedback"** es una variable que indica el puntaje de retroalimentación promedio que cada empleado recibe de sus compañeros y supervisor en la evaluación anual de 360 grados.Las puntuaciones posibles van desde 0 (la más baja) a 4 (la más alta), **"Salary"** hace referencia al sueldo base anual de los empleados, en promedio el salario de los empleado es de 73.673, **"NPS"** es el puntaje neto del promotor (NPS) es un indicador clave de la satisfacción y lealtad del cliente, es una variable que va desde 1 hasta 10 con el fin de darle un valor a dicha satisfacción. Así mismo, como variable de referencia y para el pronóstico del modelo su usó la variable **"College"** la cual indica si el empleado tiene o no un título universitario, en la base de datos original está fue registrado con las palabras "Yes" or "No" pero para términos del ejercicio se cambiaron a aquellos que NO cuentan con un título universitario (college) se identifican con el número 0 y aquellos que SÍ cuentan con un título universitario (college) con el número 1

# **Análisis de resultados**
Después de comprender el tipo de base de datos que estamos trabajando, podemos evaluar el modelo y resultados:

Los datos del archivo TechSales_Reps contienen registros de 21990 representantes de ventas de los grupos de productos de hardware y software de una empresa de alta tecnología, estos se encuentran distribuidos en 17520 (79,67%) empleados con título univeristario y 4470 (20,32%) sin un título universitario. Para cada empleado, los datos incluyen información sociodemográﬁca y educativa, para realizar el ejercicio se usaron las ocho variables descritas anteriormente.

En el modelo de árboles de decisión se puede observar que el modelo alcanzó una exactitud cercana al 80% (79,8%) destacándose el comportamiento del modelo frente a la capacidad que tiene de predecir los empleados que presentan un título universitario (college) con una sensibilidad cercana al 100% (99%). Así mismo, se ve la buena funcionalidad del modelo con una tasa de error del 20%, respaldada por su precisión del 80% que demuestra la buena capacidad predictiva del modelo.

Por otro lado, para la realización del pronóstico se uso una hoja diferente de la base de datos que contiene los empleados a evaluar, en total esta hoja cuenta con 16 empleados, de los cuales 0 (0%) NO tienen un título universitario, y 16 (100%) SÍ tienen un título universitario dada las variables evaluadas para lograr el resultado. El árbol de decisión muestra que el salario es la primera división para determinar su nivel educativo (si lo consideramos de arriba hacia abajo), con un Salary ≤ 64,100 que se toma como primer medida para encontrar las reglas del negocio. Otras variables como la edad, la retroalimentación (Feedback) y la cantidad de certificaciones también influyen en el resultado del pronóstico. La falta de divisiones directas para variables como Business, Female o Years sugiere que el modelo no las considera relevantes. Una razón que explica por qué el modelo predijo que las 16 personas en el conjunto de prueba tenían título universitario es su especificidad del 2% indicando que el modelo no es tan bueno pronosticando aquellos empleados sin título universitarios, esto se puede deber a distintas razones que se podrían evaluar a profundidad, pero una posible respuesta es por un sesgo en su entrenamiento. Es posible que en la base de datos exista muchos más empleados con títulos universitarios que sin él lo que podría hacer que el modelo "aprendiera" que la mayoría de las personas tienen un título y por tanto, repitiera ese patrón en la predicción.

Con lo anterior, podemos tener una idea más clara de como el modelo de Árboles de decisión es aplicado a una base de datos que indica las ventas de una empresa para así tomar decisiones basadas en los resultados obtenidos, que puedan ser precisos y acertados según los datos presentados.